<a href="https://colab.research.google.com/github/demichie/PhD_ModelingCourse/blob/main/Lecture4/Lecture4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Global imports for Lecture 4
import numpy as np
import matplotlib.pyplot as plt

# Plotting style (optional)
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## Lecture 4: Linear Advection and Numerical Schemes

### Part 1 (Continued): The 1D Linear Advection Equation

We derived the 1D Linear Advection Equation under several simplifying assumptions (constant density, no diffusion, no sources, 1D constant velocity flow):

$$ \frac{\partial \phi}{\partial t} + u \frac{\partial \phi}{\partial x} = 0 $$

Where:
*   $\phi(x,t)$ is the quantity being advected (e.g., concentration).
*   $u$ is the constant advection velocity.

This is a first-order, linear, **hyperbolic PDE**. Hyperbolic PDEs describe phenomena where information propagates at a finite speed and in specific directions. This contrasts with parabolic PDEs (like the heat equation), where disturbances are felt (in principle) instantly everywhere.

#### Exact Solution and Characteristic Curves

Given an initial condition $\phi(x, t=0) = \phi_0(x)$, the exact solution to the linear advection equation is:

$$ \phi(x,t) = \phi_0(x - ut) $$

This means the initial profile $\phi_0(x)$ simply **translates (shifts) without changing shape** along the x-axis with velocity $u$.
*   If $u > 0$, the profile moves to the right.
*   If $u < 0$, the profile moves to the left.

The lines $x - ut = \text{constant}$ are called **characteristic curves**, and the value of $\phi$ remains constant along these curves.

Let's visualize this exact solution for a sample initial condition.

**Initial Conditions We Will Use in Our Python Examples:**

We will often test our numerical schemes with specific initial shapes for $\phi_0(x)$:

1.  **Gaussian Pulse:** A smooth, bell-shaped curve.
    $$ \phi_0(x) = A \exp\left(-\frac{(x - x_c)^2}{2\sigma^2}\right) $$
    where $A$ is the amplitude, $x_c$ is the center of the pulse, and $\sigma$ controls its width.

2.  **Square Wave (or "Top-Hat"):** A discontinuous profile, useful for testing how schemes handle sharp gradients.
    
    \begin{equation}
    \phi_0(x) =
    \begin{cases}
    \phi_{high} & \text{if } x_{left} \leq x \leq x_{right} \\
    \phi_{low} & \text{otherwise}
    \end{cases}
    \end{equation}
    
    For example, $\phi_{high}=1$ and $\phi_{low}=0$.

In [ ]:
# --- Analytical Solution Visualization ---

def initial_condition_gaussian(x, x0=0.5, sigma=0.1):
    """A Gaussian pulse as an initial condition."""
    return np.exp(-0.5 * ((x - x0) / sigma)**2)

def initial_condition_square_wave(x, x_start=0.25, x_end=0.75):
    """A square wave (top-hat) as an initial condition."""
    phi0 = np.zeros_like(x)
    phi0[(x >= x_start) & (x <= x_end)] = 1.0
    return phi0

# Domain and parameters for analytical solution
L_domain = 2.0  # Length of the spatial domain
nx_analytical = 201 # Number of points for plotting analytical solution
x_analytical = np.linspace(0, L_domain, nx_analytical)
u_velocity = 1.0    # Advection velocity

# Choose initial condition type
# ic_func = initial_condition_gaussian
ic_func = initial_condition_square_wave

# Initial profile
phi_0_analytical = ic_func(x_analytical)
# For Gaussian specific parameters:
# phi_0_analytical = initial_condition_gaussian(x_analytical, x0=0.5, sigma=0.08)


# Time points for plotting
t_initial = 0.0
t_final_analytical = 0.5

# Analytical solution at t_final
# phi(x,t) = phi_0(x - ut)
# To implement this, we evaluate phi_0 at the "shifted back" coordinates
x_shifted_analytical = x_analytical - u_velocity * t_final_analytical
phi_t_final_analytical = ic_func(x_shifted_analytical)
# For Gaussian:
# phi_t_final_analytical = initial_condition_gaussian(x_shifted_analytical, x0=0.5, sigma=0.08)


# Plotting
plt.figure(figsize=(10, 6))
plt.plot(x_analytical, phi_0_analytical, 'b-', label=f'Initial Condition $\phi_0(x)$ (t={t_initial:.1f})')
plt.plot(x_analytical, phi_t_final_analytical, 'r--', label=f'Exact Solution $\phi(x,t)$ (t={t_final_analytical:.1f}, u={u_velocity})')
plt.xlabel('Position x')
plt.ylabel('$\phi$')
plt.title('Exact Solution of 1D Linear Advection')
plt.legend()
plt.grid(True)
plt.ylim(-0.1, 1.1) # Adjust if using Gaussian with peak > 1 or < 0
plt.show()

## Part 2: Boundary Conditions

(Recap of Dirichlet and Neumann omitted here for brevity - assume it's covered in Markdown like slides)

### Periodic Boundary Conditions
For a domain $x \in [0, L]$:
*   $\phi(0, t) = \phi(L, t)$
*   Numerically, for grid points $i=0, \dots, N_x-1$:
    *   Value "left" of $x_0$ (e.g., $\phi_{-1}^n$) is taken from $\phi_{N_x-1}^n$.
    *   Value "right" of $x_{N_x-1}$ (e.g., $\phi_{N_x}^n$) is taken from $\phi_{0}^n$.

Periodic BCs are useful for testing schemes as they allow features to pass through boundaries without artificial reflections, isolating the scheme's intrinsic behavior.

## Part 3: Finite Difference Schemes for Advection

We will now develop numerical schemes by discretizing the 1D linear advection equation:
$$ \frac{\partial \phi}{\partial t} + u \frac{\partial \phi}{\partial x} = 0 $$

### Time Discretization: Forward Euler
We use Forward Euler for the time derivative:
$$ \frac{\partial \phi}{\partial t}\bigg|_i^n \approx \frac{\phi_i^{n+1} - \phi_i^n}{\Delta t} $$
This is $\mathcal{O}(\Delta t)$.
Our goal is to find $\phi_i^{n+1}$ based on values at time level $n$.

---
### Scheme 1: Forward Time, Centered Space (FTCS)
We combine Forward Euler in time with a Central Difference in space:
$$ \frac{\phi_i^{n+1} - \phi_i^n}{\Delta t} + u \frac{\phi_{i+1}^n - \phi_{i-1}^n}{2\Delta x} = 0 $$
Update formula:
$$ \phi_i^{n+1} = \phi_i^n - \frac{u \Delta t}{2\Delta x}(\phi_{i+1}^n - \phi_{i-1}^n) $$
This scheme has a truncation error of $\mathcal{O}(\Delta t, \Delta x^2)$.

Let $C = \frac{u \Delta t}{\Delta x}$ be the **Courant number**. Then:
$$ \phi_i^{n+1} = \phi_i^n - \frac{C}{2}(\phi_{i+1}^n - \phi_{i-1}^n) $$

In [ ]:
# Jupyter Notebook Code Cell: FTCS Scheme for 1D Linear Advection (Revised)

# Import necessary libraries (if not already imported globally)
import numpy as np
import matplotlib.pyplot as plt

print("--- FTCS Scheme for 1D Linear Advection ---")

# --- Helper function for initial condition ---
def initial_condition_square_wave(x, L, x_start_ratio=0.15, x_end_ratio=0.35, A=1.0, base_val=0.0):
    """
    Creates a square wave initial condition, aware of periodic domain [0, L).
    x: array of grid points
    L: domain length
    x_start_ratio: starting position of the wave as a fraction of L
    x_end_ratio: ending position of the wave as a fraction of L
    A: amplitude of the wave
    base_val: base value of the wave
    """
    phi0 = np.full_like(x, base_val)
    x_start_abs = x_start_ratio * L
    x_end_abs = x_end_ratio * L

    # Condition for points within the main pulse
    condition = (x >= x_start_abs) & (x <= x_end_abs)
    phi0[condition] = A

    # Handle cases where the pulse wraps around due to periodicity
    # If start > end (meaning it wraps around from L to 0)
    if x_start_abs > x_end_abs: # e.g. starts at 0.8L, ends at 0.2L
        condition_wrap = (x >= x_start_abs) | (x <= x_end_abs)
        phi0[condition_wrap] = A
    return phi0

def analytical_solution_periodic(x_grid, t, u, L, phi0_func, **phi0_kwargs):
    """
    Calculates the analytical solution for 1D linear advection with periodic BCs.
    phi0_func is the function defining the initial condition phi_0(x_prime).
    **phi0_kwargs are any additional arguments needed by phi0_func (like L for square wave).
    """
    # Calculate the effective 'original' x positions from which values were advected
    x_effective_source = (x_grid - u * t) % L
    # The modulo operator handles the periodic wrapping naturally.
    # Example: if L=1, x=0.1, u=1, t=0.3 -> x_eff_src = (0.1 - 0.3) % 1 = -0.2 % 1 = 0.8
    # Example: if L=1, x=0.8, u=1, t=0.3 -> x_eff_src = (0.8 - 0.3) % 1 = 0.5 % 1 = 0.5
    # Example: if L=1, x=0.1, u=1, t=1.3 -> x_eff_src = (0.1 - 1.3) % 1 = -1.2 % 1 = 0.8

    return phi0_func(x_effective_source, L=L, **phi0_kwargs)


# --- 1. Simulation Parameters ---
L = 1.0         # Length of the spatial domain
nx = 101        # Number of spatial grid points
dx = L / (nx -1) if nx > 1 else L # Spatial step size, avoid division by zero if nx=1
# For periodic, often dx = L / nx and x_grid = np.linspace(0, L-dx, nx)
# Let's stick to linspace(0,L,nx) for now as it's common for FD node-based
u = 1.0         # Advection velocity

# Time-stepping parameters
CFL = 0.4       # Courant number (FTCS is unstable, but we set a value)
dt = CFL * dx / abs(u) if abs(u) > 1e-9 else 1e-3 # Avoid division by zero if u=0
# t_final = 0.01   # For FTCS, a very short time is needed to see instability before NaNs
t_final = 0.05   # Let's try a slightly longer time to ensure instability is clear
n_steps = int(t_final / dt) if dt > 0 else 0

print(f"Domain length: {L}, Number of points: {nx}, dx: {dx:.4f}")
print(f"Velocity: {u}, CFL: {CFL:.2f}, dt: {dt:.4e}, Final time: {t_final:.3f}, Steps: {n_steps}")

# --- 2. Grid and Initial Condition ---
x_grid = np.linspace(0, L, nx, endpoint=True) # Includes L, adjust if using L/nx for dx
if np.isclose(x_grid[-1],L) and nx > 1 : # If endpoint is L, for periodic, last point often overlaps first
    x_grid_periodic = np.linspace(0,L-dx,nx-1) # A common periodic grid excludes L
    # However, for simplicity with explicit BC loop, we'll use 0..nx-1 indices on a grid that includes L
    # and the periodic BC will map nx-1's right neighbor to 0, and 0's left to nx-1.
    # If nx is number of cells, then dx = L/nx, x_grid = dx/2 + np.arange(nx)*dx (cell centers)
    # Sticking with node-based linspace(0,L,nx) for now.

# Initial condition: A Square wave
phi_initial = initial_condition_square_wave(x_grid, L, x_start_ratio=0.15, x_end_ratio=0.4, A=1.0)


# Analytical solution at t_final
phi_analytical_final = analytical_solution_periodic(x_grid, t_final, u, L,
                                                   initial_condition_square_wave,
                                                   x_start_ratio=0.15, x_end_ratio=0.4, A=1.0)


# --- 3. Initialization ---
phi = phi_initial.copy()    # Current solution array
phi_new = np.zeros_like(phi) # Array for the solution at the next time step

# --- 4. Time-stepping Loop (FTCS with Periodic BCs) ---
if n_steps > 0: # Only run if there are steps to take
    for n_loop_idx in range(n_steps):
        phi_old_step_for_rhs = phi.copy() # Use values from time level 'n' for RHS

        for i_node in range(nx): # Loop over ALL physical grid points 0 to nx-1
            # Determine periodic indices for neighbors
            # Left neighbor index (im1 for i-1)
            if i_node == 0:
                im1 = nx - 1
            else:
                im1 = i_node - 1

            # Right neighbor index (ip1 for i+1)
            if i_node == nx - 1:
                ip1 = 0
            else:
                ip1 = i_node + 1

            # FTCS formula using values from phi_old_step_for_rhs
            phi_new[i_node] = phi_old_step_for_rhs[i_node] - \
                              u * dt / (2 * dx) * \
                              (phi_old_step_for_rhs[ip1] - phi_old_step_for_rhs[im1])

        # Update solution for the next time step
        phi = phi_new.copy()
else:
    print("n_steps is 0 or less. No simulation run. Final solution is initial condition.")
    phi = phi_initial.copy() # Ensure phi is the initial if no steps


# --- 5. Plotting Results ---
plt.figure(figsize=(10, 6))
plt.plot(x_grid, phi_initial, label='Initial Condition (t=0)', linestyle=':', color='gray')
plt.plot(x_grid, phi, label=f'FTCS Numerical (t={t_final:.3f}, CFL={CFL:.2f})', marker='.', markersize=4, linestyle='-')
plt.plot(x_grid, phi_analytical_final, label=f'Analytical (t={t_final:.3f})', linestyle='--', color='red')

plt.title(f'1D Linear Advection with FTCS Scheme (Periodic BCs)')
plt.xlabel('Spatial domain x')
plt.ylabel('$\phi(x,t)$')
plt.legend()
plt.grid(True)

# Adjust ylim to better see instability if it occurs
min_val = min(phi_initial.min(), phi.min(), phi_analytical_final.min())
max_val = max(phi_initial.max(), phi.max(), phi_analytical_final.max())
padding = 0.2 * (max_val - min_val) if (max_val - min_val) > 1e-6 else 0.2
y_bottom = min_val - padding
y_top = max_val + padding
if np.any(np.abs(phi) > 2*np.max(np.abs(phi_initial))): # If solution blows up
     y_bottom = min(y_bottom, -2*np.max(np.abs(phi_initial)))
     y_top = max(y_top, 2*np.max(np.abs(phi_initial)))

plt.ylim(y_bottom, y_top)
plt.show()

# Print a note about expected FTCS behavior
print("\nNote: The FTCS scheme is expected to be unstable for the pure advection equation.")
print("If the plot shows large oscillations or NaN values, this demonstrates the instability.")
if n_steps > 0 and (np.any(np.isnan(phi)) or np.any(np.abs(phi) > 5*np.max(np.abs(phi_initial)) if np.max(np.abs(phi_initial)) > 0 else 10 )):
    print("Instability detected in the FTCS solution!")
elif n_steps == 0:
    print("No time steps were run.")
else:
    print("FTCS simulation completed. Check plot for stability.")

---
### Scheme 2: Upwind Schemes
Upwind schemes use one-sided differences based on the direction of flow $u$.

#### Upwind for $u > 0$: Forward Time, Backward Space (FTBS)
If $u > 0$, information comes from the left.
$$ \frac{\phi_i^{n+1} - \phi_i^n}{\Delta t} + u \frac{\phi_i^n - \phi_{i-1}^n}{\Delta x} = 0 $$
Update formula:
$$ \phi_i^{n+1} = \phi_i^n - C(\phi_i^n - \phi_{i-1}^n) $$
where $C = u \Delta t / \Delta x$. This is $\mathcal{O}(\Delta t, \Delta x)$.

In [ ]:
# --- Upwind (FTBS for u > 0) Scheme Implementation and Test ---

# Simulation parameters (can reuse L, nx, u, CFL, t_final from FTCS or define new)
L_upwind = 1.0
nx_upwind = 101
dx_upwind = L_upwind / (nx_upwind - 1)
x_upwind = np.linspace(0, L_upwind, nx_upwind)

u_upwind = 1.0        # Advection velocity (MUST BE > 0 for this FTBS)
CFL_upwind = 0.8      # Courant number (should be <= 1 for stability)
dt_upwind = CFL_upwind * dx_upwind / u_upwind
t_final_upwind = 0.5
nt_upwind = int(t_final_upwind / dt_upwind)

print(f"\nUpwind (FTBS) Parameters: nx={nx_upwind}, dx={dx_upwind:.4f}, dt={dt_upwind:.4f}, Courant No. C={CFL_upwind:.2f}")

# Initial condition
# phi_current_upwind = initial_condition_gaussian(x_upwind, x0=0.25, sigma=0.05)
phi_current_upwind = initial_condition_square_wave(x_upwind, x_start=0.1, x_end=0.4)
phi_initial_upwind = phi_current_upwind.copy()

# Time-stepping loop for Upwind (FTBS)
phi_new_upwind = np.zeros_like(phi_current_upwind)

for n in range(nt_upwind):
    phi_old_upwind = phi_current_upwind.copy()

    # Periodic Boundary Conditions
    for i in range(nx_upwind): # Loop through all points including boundaries
        # Determine index for phi[i-1] with periodicity
        im1 = (i - 1 + nx_upwind) % nx_upwind # Modulo operator handles wrap-around

        phi_new_upwind[i] = phi_old_upwind[i] - \
                            CFL_upwind * (phi_old_upwind[i] - phi_old_upwind[im1])

    phi_current_upwind = phi_new_upwind.copy()

# Analytical solution for comparison
x_shifted_upwind = (x_upwind - u_upwind * t_final_upwind)
x_shifted_upwind_periodic = np.mod(x_shifted_upwind, L_upwind)
# phi_analytical_upwind = initial_condition_gaussian(x_shifted_upwind_periodic, x0=0.25, sigma=0.05)
phi_analytical_upwind = initial_condition_square_wave(x_shifted_upwind_periodic, x_start=0.1, x_end=0.4)


# Plotting Upwind (FTBS) Result
plt.figure(figsize=(10, 6))
plt.plot(x_upwind, phi_initial_upwind, 'k-', label='Initial Condition')
plt.plot(x_upwind, phi_analytical_upwind, 'g--', label='Analytical Solution')
plt.plot(x_upwind, phi_current_upwind, 'b.-', label=f'Upwind (FTBS, CFL={CFL_upwind:.1f})')
plt.xlabel('Position x')
plt.ylabel('$\phi$')
plt.title(f'Upwind (FTBS) Scheme for Advection (t={t_final_upwind:.2f}) - Numerical Diffusion')
plt.legend()
plt.grid(True)
plt.ylim(-0.1, 1.1)
plt.show()
# Corresponds to FIGURES/FTBS_result_diffusion.png

#### Upwind for $u < 0$: Forward Time, Forward Space (FTFS)
If $u < 0$, information comes from the right.
$$ \frac{\phi_i^{n+1} - \phi_i^n}{\Delta t} + u \frac{\phi_{i+1}^n - \phi_i^n}{\Delta x} = 0 $$
Update formula:
$$ \phi_i^{n+1} = \phi_i^n - C(\phi_{i+1}^n - \phi_i^n) $$
This is $\mathcal{O}(\Delta t, \Delta x)$.
(L'implementazione Python sarebbe simmetrica a FTBS, usando `ip1 = (i + 1) % nx_upwind`.)

---
### Numerical Artifacts: Diffusion and Dispersion
*   **Numerical Diffusion (Dissipation):** Smears sharp gradients, reduces peak amplitudes. Seen in Upwind and Lax-Friedrichs.
*   **Numerical Dispersion:** Different wavelengths travel at incorrect speeds, causing spurious oscillations (wiggles). Can occur with centered or higher-order schemes.


### Visualizing Advection: Animating the Upwind Scheme

Static plots show the solution at specific time points (e.g., initial and final). However, advection is a dynamic process. An animation can provide a much better intuition for how the profile evolves over time and how numerical artifacts like diffusion accumulate.

**Idea Behind Creating an Animation:**

1.  **Store Snapshots:** During the time-stepping loop of our numerical simulation, we need to save the entire spatial profile $\phi(x)$ at regular intervals (e.g., every `M` time steps).
2.  **Iterate and Plot:** After the simulation, we create a plotting function that takes a "frame number" (corresponding to a saved snapshot) as input. This function clears the previous plot and draws the $\phi(x)$ profile for that specific frame/time.
3.  **Animation Tool:** Python's `matplotlib` library has an `animation` module (specifically `FuncAnimation`) that repeatedly calls our plotting function for each saved snapshot, creating the frames of the animation.
4.  **Display/Save:** The animation can then be displayed directly in the notebook or saved as a GIF or video file.

We won't delve into the intricate details of `matplotlib.animation` syntax here, but the core is to generate a sequence of plots that, when shown rapidly, create the illusion of motion.

In [ ]:
# Jupyter Notebook Code Cell: Animating the Upwind Scheme (Revised)

# Import animation capabilities from matplotlib
from matplotlib.animation import FuncAnimation
from IPython.display import HTML # To display animation in Jupyter

print("--- Animating 1D Linear Advection with Upwind Scheme (Revised) ---")

# --- 1. Simulation Parameters (can reuse from static Upwind or redefine) ---
L_anim = 1.0
nx_anim = 101
dx_anim = L_anim / (nx_anim - 1) # dx_anim is L_anim / (nx_anim-1) if nx_anim includes endpoints
# If nx_anim is number of cells, dx_anim = L_anim / nx_anim and x_grid is cell centers
u_anim = 1.0 # Try positive and negative

CFL_anim = 0.8
dt_anim = CFL_anim * dx_anim / abs(u_anim if u_anim != 0 else 1e-9) # Avoid division by zero if u_anim is 0
t_final_anim = 2.0  # Simulate for e.g., two full domain traversals if L=1, u=1
n_steps_anim = int(t_final_anim / dt_anim) if dt_anim > 0 else 0


print(f"Animation - dx: {dx_anim:.4f}, dt: {dt_anim:.4e}, CFL: {CFL_anim:.2f}, Steps: {n_steps_anim}")

# --- 2. Grid and Initial Condition ---
# x_grid_anim = np.linspace(0, L_anim, nx_anim) # Includes endpoints
# If using cell centers, it would be:
# x_grid_anim = np.linspace(dx_anim/2, L_anim - dx_anim/2, nx_anim)
x_grid_anim = np.arange(0, L_anim, dx_anim) # This creates nx_anim points if L_anim/dx_anim is integer
if not np.isclose(x_grid_anim[-1] + dx_anim, L_anim): # Adjust if L_anim is not perfectly covered
     x_grid_anim = np.linspace(0, L_anim - dx_anim, nx_anim) # Common for periodic, excluding endpoint L

nx_anim = len(x_grid_anim) # Update nx_anim based on the actual grid generated

# Initial condition: Gaussian pulse
A_anim = 1.0
x_center_anim = 0.25  # Center of the initial pulse
sigma_anim = 0.05     # Width of the pulse

# To make the Gaussian periodic aware from the start if it's near boundaries
# This handles the case where the initial pulse itself wraps around
phi_initial_anim = np.zeros_like(x_grid_anim)
for i_offset in [-1, 0, 1]: # Check current, left-wrapped, right-wrapped domain
    center_offset = x_center_anim + i_offset * L_anim
    phi_initial_anim += A_anim * np.exp(-(x_grid_anim - center_offset)**2 / (2 * sigma_anim**2))


# --- 3. Initialization for Upwind ---
phi_anim = phi_initial_anim.copy()
phi_new_anim = np.zeros(nx_anim)

# --- 4. Store solutions for animation ---
animation_frame_interval = max(1, n_steps_anim // 100 if n_steps_anim > 0 else 1)
print(f"Saving a frame for animation every {animation_frame_interval} steps.")

solutions_for_animation = []

# --- 5. Time-stepping Loop (Upwind with Periodic BCs) ---
for n in range(n_steps_anim + 1):
    if n % animation_frame_interval == 0:
        solutions_for_animation.append(phi_anim.copy())

    if n == n_steps_anim:
        break

    phi_old_step_anim = phi_anim.copy()

    if u_anim >= 0: # FTBS for u >= 0 (includes u=0 case, no change)
        for i in range(nx_anim):
            im1 = (i - 1 + nx_anim) % nx_anim
            phi_new_anim[i] = phi_old_step_anim[i] - \
                              u_anim * dt_anim / dx_anim * (phi_old_step_anim[i] - phi_old_step_anim[im1])
    else: # FTFS for u < 0
        for i in range(nx_anim):
            ip1 = (i + 1) % nx_anim
            phi_new_anim[i] = phi_old_step_anim[i] - \
                              u_anim * dt_anim / dx_anim * (phi_old_step_anim[ip1] - phi_old_step_anim[i])

    phi_anim = phi_new_anim.copy()

print(f"Simulation for animation finished. Stored {len(solutions_for_animation)} frames.")

# --- 6. Create and Display Animation ---
fig_anim, ax_anim = plt.subplots(figsize=(8, 5))

# Plot initial condition (fixed)
ax_anim.plot(x_grid_anim, phi_initial_anim, 'g:', lw=1.5, label='Initial Condition')

# Line for the numerical solution (will be updated)
line_anim, = ax_anim.plot(x_grid_anim, solutions_for_animation[0], 'b-', lw=2, label='Upwind Numerical')

# Line for the analytical solution (will be updated)
analytical_line_anim, = ax_anim.plot(x_grid_anim, phi_initial_anim, 'r--', lw=1.5, label='Analytical')

ax_anim.set_xlim(0, L_anim)
ax_anim.set_ylim(min(0, phi_initial_anim.min()) - 0.2 * A_anim, phi_initial_anim.max() + 0.2 * A_anim)
ax_anim.set_xlabel("Spatial domain x")
ax_anim.set_ylabel("$\phi(x,t)$")
ax_anim.set_title("1D Linear Advection with Upwind Scheme")
time_text_anim = ax_anim.text(0.05, 0.90, '', transform=ax_anim.transAxes, fontsize=10)
cfl_text_anim = ax_anim.text(0.05, 0.82, f'CFL = {CFL_anim:.2f}', transform=ax_anim.transAxes, fontsize=10)
ax_anim.legend(loc='upper right', fontsize=9)
ax_anim.grid(True)

# Function to update the plot for each frame
def update_animation(frame_num):
    numerical_solution_this_frame = solutions_for_animation[frame_num]
    line_anim.set_ydata(numerical_solution_this_frame)

    current_time = frame_num * animation_frame_interval * dt_anim
    time_text_anim.set_text(f'Time = {current_time:.2f} s\nFrame = {frame_num}')

    # Update analytical solution for this frame's time, considering periodicity
    # The argument to the initial condition function phi_0 is (x - ut)
    # We need to map (x_grid_anim - u_anim * current_time) to the [0, L_anim) interval periodically

    # Effective x for analytical solution, wrapped into [0, L_anim)
    x_analytical_effective = (x_grid_anim - u_anim * current_time) % L_anim

    # Now, evaluate the initial condition shape at these effective x values.
    # This assumes phi_0 was defined on [0, L_anim). If phi_0 was centered at x_c,
    # the argument to the exponential becomes (x_analytical_effective - x_center_anim)
    # However, it's easier to reconstruct the full periodic analytical solution:

    phi_analytical_current = np.zeros_like(x_grid_anim)
    for i_offset in [-2, -1, 0, 1, 2]: # Consider multiple wrappings for smoothness of analytical
        center_offset = x_center_anim + u_anim * current_time + i_offset * L_anim
        phi_analytical_current += A_anim * np.exp(-(x_grid_anim - center_offset)**2 / (2 * sigma_anim**2))

    analytical_line_anim.set_ydata(phi_analytical_current)

    return line_anim, analytical_line_anim, time_text_anim

# Create the animation
if solutions_for_animation: # Check if there are frames to animate
    ani = FuncAnimation(fig_anim, update_animation, frames=len(solutions_for_animation),
                        interval=70, blit=True) # Increased interval for smoother perception
    plt.close(fig_anim)
    display(HTML(ani.to_jshtml()))
else:
    print("No frames stored for animation (check n_steps_anim and animation_frame_interval).")
    plt.show() # Show the static plot setup if no animation

# Optional: Save as GIF
# if solutions_for_animation:
#     try:
#         ani.save('upwind_advection_periodic.gif', writer='pillow', fps=15)
#         print("Animation saved as upwind_advection_periodic.gif")
#     except Exception as e:
#         print(f"Could not save animation: {e}. Ensure pillow or imagemagick is installed.")

### Exercises: Exploring the Upwind Scheme with Animation

Now that you have the code to animate the Upwind scheme, try experimenting with it:

1.  **Vary the CFL Number:**
    *   In the animation code cell, change `CFL_anim` to values like `0.2`, `0.5`, `0.99`, and `1.0`. How does the numerical diffusion (smearing of the pulse) change with CFL?
    *   What happens if you set `CFL_anim` to a value slightly greater than 1 (e.g., `1.05` or `1.1`)? Observe the animation carefully. Does the scheme remain stable? (The Upwind scheme should become unstable for CFL > 1).

2.  **Impact of Spatial Resolution ($\Delta x$):**
    *   Keep `CFL_anim` fixed (e.g., at `0.8`).
    *   Change `nx_anim` to a smaller value (e.g., `51`, coarser grid) and then to a larger value (e.g., `201`, finer grid). Remember that `dt_anim` will change if CFL is fixed.
    *   How does the numerical diffusion change with grid resolution for a fixed CFL? Does a finer grid always produce a "sharper" (less diffused) result at the same *physical time*?

3.  **Long-Term Advection (Multiple Periods):**
    *   Set `t_final_anim` such that the pulse should theoretically pass through the periodic domain multiple times (e.g., if $L=1, u=1$, set $t_{final}=2.0$ or $3.0$).
    *   Observe how the numerical diffusion accumulates over time. Does the pulse degrade significantly after several traversals?

4.  **Different Initial Conditions:**
    *   Modify `phi_initial_anim` to be a **square wave** (top-hat function) instead of a Gaussian.
        ```python
        # Example for square wave:
        # phi_initial_anim = np.zeros(nx_anim)
        # start_index = int(0.15 / dx_anim)
        # end_index = int(0.35 / dx_anim)
        # phi_initial_anim[start_index:end_index+1] = 1.0
        ```
    *   How does the Upwind scheme handle the sharp discontinuities of the square wave? Observe the smearing of the edges.

5.  **(Challenge) Mass Conservation Check with Animation:**
    *   Modify the animation loop to calculate the total "mass" (integral of $\phi$) at each animation frame: `total_phi = np.sum(phi_anim) * dx_anim`.
    *   Plot this `total_phi` as a function of time (either in a separate plot or by adding text to the animation).
    *   Does the Upwind scheme with periodic BCs conserve mass well? (It should, to a good approximation).

These exercises will help you build a stronger intuition for the behavior of the Upwind scheme and the impact of numerical parameters.